In [17]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from missingpy import MissForest

from sklearn.impute import KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.svm import  SVR

In [2]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

X_train = X_train.set_index("id")
X_test = X_test.set_index("id")
y_train = y_train.set_index("id")

In [3]:
X_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,785176.201298,...,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1.0,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,...,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,...,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3.0,11.642076,NaN,NaN,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,785176.254867,...,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.321063,1.085442e+06,NaN,102746.516920
4.0,10.407121,832442.831424,20585.557007,NaN,9.957182e+05,8.419164,597900.423639,12894.065081,1.063199e+06,785176.190880,...,1.031009e+06,-1025.223865,18348.460040,NaN,NaN,1230.088215,10.250096,1.024812e+06,101815.745499,105163.749149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207.0,9.895208,832442.883343,20585.510774,1082.657477,1.107019e+06,10.915545,597900.459438,NaN,1.020097e+06,785176.241638,...,1.004860e+06,-999.439794,17689.388714,10.472291,9.792557,1249.098796,10.939983,1.092890e+06,97402.723818,107110.572397
1208.0,9.838322,832442.816133,20585.530631,1020.816077,9.834291e+05,NaN,597900.428561,10842.043309,1.110318e+06,785176.275709,...,1.080810e+06,-1113.072752,15994.070051,10.269221,9.771282,1402.080672,10.958833,1.083262e+06,107100.133295,109984.169649
1209.0,9.267322,832442.815687,20585.525811,1076.663108,7.651645e+05,9.579855,NaN,NaN,1.092330e+06,785176.241429,...,1.047013e+06,-482.079954,11539.602821,10.346105,10.361797,965.008332,10.483549,1.007871e+06,119139.273272,105240.374580


In [4]:
X_train, y_train = shuffle(X_train, y_train, random_state=0)

In [5]:
X_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
198.0,NaN,832442.835548,20585.572205,1037.772597,NaN,9.765165,597900.432429,11528.028108,1.034177e+06,785176.234641,...,1.069972e+06,-1102.466322,15766.208756,10.029853,10.247402,1050.047922,9.466595,1.041488e+06,110086.347597,108926.755734
1009.0,10.144704,832442.824862,20585.562924,1035.817825,1.068988e+06,11.549407,NaN,8405.098442,1.121805e+06,785176.215455,...,1.026280e+06,-821.858938,10840.260860,NaN,11.053307,872.059294,9.271999,1.060668e+06,NaN,103446.109646
55.0,9.560128,832442.907929,20585.496577,1094.254179,1.101507e+06,10.297179,NaN,10736.058521,NaN,785176.187143,...,1.074375e+06,-908.613692,14219.663267,10.262830,9.207310,1347.087944,10.069470,1.095932e+06,91292.245016,109452.435128
743.0,10.578523,832442.873029,20585.563095,1011.389786,1.102438e+06,10.429502,597900.425893,10422.097698,NaN,785176.182204,...,1.032776e+06,-962.715491,14900.485999,10.403145,10.788195,773.096326,10.164620,1.061034e+06,89463.166281,108766.786602
907.0,10.404152,NaN,20585.486909,1004.339873,1.269424e+06,8.114029,597900.449316,8170.017487,8.673935e+05,785176.185196,...,1.073815e+06,-702.251500,11792.230531,10.882244,10.902264,585.075225,10.340209,1.074091e+06,93561.050071,100433.230379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033.0,NaN,832442.849817,20585.483603,1095.559877,1.063630e+06,NaN,NaN,8945.009308,1.063815e+06,785176.201947,...,1.070712e+06,-879.413638,13152.422962,NaN,10.560826,875.057873,10.492375,1.033628e+06,99202.051520,102071.547199
763.0,10.529848,832442.840957,20585.473817,1073.654698,1.145325e+06,10.434580,597900.454931,12406.064710,1.194430e+06,785176.207422,...,1.032234e+06,-1087.614176,17667.480867,10.733368,9.586136,1212.060782,11.098854,1.052828e+06,94158.410790,NaN
835.0,9.754108,832442.894150,20585.508755,1076.854352,1.037915e+06,9.616252,597900.461438,11300.019091,8.684131e+05,785176.253301,...,1.027375e+06,-661.119561,8009.142281,10.649794,9.661812,1104.061126,8.735388,1.099408e+06,110362.558344,102136.601159


In [6]:
y_train

,y
id,
198.0,61.0
1009.0,78.0
55.0,80.0
743.0,62.0
907.0,66.0
...,...
1033.0,67.0
763.0,56.0
835.0,87.0


In [7]:
# convert to np arrays
X_train = X_train.iloc[:,:].to_numpy()
y_train = y_train.iloc[:,:].to_numpy().reshape(-1)
X_test = X_test.iloc[:,:].to_numpy()

In [8]:
X_train

array([[           nan, 8.32442836e+05, 2.05855722e+04, ...,
        1.04148824e+06, 1.10086348e+05, 1.08926756e+05],
       [1.01447037e+01, 8.32442825e+05, 2.05855629e+04, ...,
        1.06066812e+06,            nan, 1.03446110e+05],
       [9.56012805e+00, 8.32442908e+05, 2.05854966e+04, ...,
        1.09593239e+06, 9.12922450e+04, 1.09452435e+05],
       ...,
       [9.75410813e+00, 8.32442894e+05, 2.05855088e+04, ...,
        1.09940811e+06, 1.10362558e+05, 1.02136601e+05],
       [9.83388021e+00, 8.32442844e+05, 2.05854942e+04, ...,
        1.05451696e+06, 9.91332513e+04, 1.04712893e+05],
       [8.61054169e+00, 8.32442875e+05, 2.05855586e+04, ...,
        1.07153478e+06, 8.88563568e+04, 1.08273943e+05]])

In [9]:
y_train

array([61., 78., 80., ..., 87., 68., 85.])

In [10]:
from collections.abc import Iterable 

class Pipeline:
    def __init__(self, pipeline_def, predefined_variables):
        self.pipeline_definitions = pipeline_definitions
        self.predefined_variables = predefined_variables
        self.best_model = {}
    
    def run_backtracked_steps_and_tuning_params(self, steps, tuning_params):
        variables = copy.deepcopy(self.predefined_variables)
        for tuning_param, value in tuning_params.items():
            variables[tuning_param] = value
        for step in steps.values():
            func_output = step["function"](*[variables[input_name] for input_name in step["input"]])
            if isinstance(func_output, list) or isinstance(func_output, tuple):
                z = zip(step["output"], func_output)
                for k, v in z:
                    variables[k] = v
            else:
                output_name = step["output"][0]
                variables[output_name] = func_output
                
        # update best model
        if self.best_model == {} or self.best_model["r2_score"] < variables["r2_score"]:
            self.best_model["steps"] = steps
            self.best_model["tuning_params"] = tuning_params
            self.best_model["r2_score"] = variables["r2_score"]
            self.best_model["y_test"] = variables["y_test"]
    
    def _backtrack_tuning_params(
        self, 
        pos,
        steps, 
        tuning_params_options,
        current_tuning_params = {}
    ):
        if pos == len(tuning_params_options.keys()):
            self.run_backtracked_steps_and_tuning_params(steps, current_tuning_params)
            return
        else:
            tuning_param_name = list(tuning_params_options.keys())[pos]
            list_values = tuning_params_options[tuning_param_name]
            for value in list_values:
                next_tuning_params = copy.deepcopy(current_tuning_params)
                next_tuning_params[tuning_param_name] = value
                self._backtrack_tuning_params(pos + 1, steps, tuning_params_options, next_tuning_params)
    
    def backtrack_tuning_params(self, steps, tuning_params_options):
        self._backtrack_tuning_params(0, steps, tuning_params_options, {})

    def _backtrack_steps(self, pos, step_names, current_steps = {}):
        if pos == len(step_names):
            tuning_params_options = {}
            for func in current_steps.values():
                if "tuning_params" in func.keys():
                    for param, list_values in func["tuning_params"].items():
                        if param in tuning_params_options:
                            raise Exception(f"Tuning parameter {param} is already defined")
                        tuning_params_options[param] = list_values
            self.backtrack_tuning_params(current_steps, tuning_params_options)
            return
        else:
            step_name = step_names[pos]
            for func in self.pipeline_definitions[step_name]:
                new_steps = copy.deepcopy(current_steps)
                new_steps[step_name] = func
                self._backtrack_steps(pos + 1, step_names, new_steps)
    
    def backtrack_steps(self):
        step_names = list(self.pipeline_definitions.keys())
        self._backtrack_steps(0, step_names, {})
    
    def run(self):
        self.backtrack_steps()
        data = {
            "id": [i * 1.0 for i in range(len(self.best_model["y_test"]))],
            "y": self.best_model["y_test"],
        }
        df = pd.DataFrame(data)
        df.to_csv("y_test.csv", index="id")

In [24]:
def scale_data(X_train, X_test, scaler):
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [12]:
def impute_missing_values_by_knn_imputer(X_train, X_test, n_neighbors):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.fit_transform(X_test)
    return X_train, X_test

In [13]:
def delete_outliers_by_isolation_forest(X, y, random_state=0):
    print(f"delete outliers by isolation forest with random_state={random_state}")
    print("old shape", X.shape, y.shape)
    clf = IsolationForest(random_state=random_state).fit(X)
    yhat = clf.predict(X)
    # select all rows that are not outliers
    mask = yhat != -1
    X, y = X[mask], y[mask]
    print("new shape", X.shape, y.shape)
    return X, y

In [14]:
def select_features_by_random_forest_regressor(X_train, y_train, X_test):
    selector = SelectFromModel(estimator=RandomForestRegressor(n_estimators = 100)).fit(X_train, y_train)
    print(f"Selected {selector.get_support().sum()}/{len(selector.get_support())} features")
    return selector.transform(X_train), selector.transform(X_test)

def select_features_by_variance_threshold(X_train, X_test):
    selector = VarianceThreshold()
    selector.fit(X_train)
    to_delete = np.where(selector.variances_ == 0)[0]
    X_train = np.delete(X_train, to_delete, axis=1)
    X_test = np.delete(X_test, to_delete, axis=1)
    return X_train, X_test

In [15]:
def run_model(model, X_train, y_train, X_test):
    r2_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="r2")
    r2_score_mean = sum(r2_scores) / len(r2_scores)
    print("r2_scores", r2_scores, "r2_score_mean", r2_score_mean)
    model.fit(X_train, y_train)
    y_test = model.predict(X_test).ravel()
    return r2_score_mean, y_test

In [21]:
pipeline_definitions = {
    "scale_data": [
          {
              "function": scale_data,
              "input": ["X_train", "X_test", "scaler"],
              "output": ["X_train", "X_test"],
              "tuning_params": {
                  "scaler": [StandardScaler(), RobustScaler()],
              },
          }
    ],
    
    "impute_missing_values": [
        {
            "function": impute_missing_values_by_knn_imputer,
            "input": ["X_train", "X_test", "n_neighbors"],
            "output": ["X_train", "X_test"],
            "tuning_params": {
                "n_neighbors": [10, 15, 20],
            }
        },
    ],

    "delete_outliers": [
        {
            "function": delete_outliers_by_isolation_forest,
            "input": ["X_train", "y_train", "random_state"],
            "output": ["X_train", "y_train"],
            "tuning_params": {
                "random_state": [None],
            }
        }
    ],
    
    "select_features": [
        {
            "function": select_features_by_random_forest_regressor,
            "input": ["X_train", "y_train", "X_test"],
            "output": ["X_train", "X_test"],
        },
        {
            "function": select_features_by_variance_threshold,
            "input": ["X_train", "X_test"],
            "output": ["X_train", "X_test"],
        }
    ],
    
    "run_model": [
        {
            "function": run_model,
            "input": ["model", "X_train", "y_train", "X_test"],
            "output": ["r2_score", "y_test"],
            "tuning_params": {
                "model": [
                    RandomForestRegressor(max_depth=10, random_state=0),
                    SVR(C=37, gamma=0.0219),
                ]
            }
        }
    ],
}

predefined_variables = {
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
}

In [22]:
pipeline = Pipeline(pipeline_definitions, predefined_variables)

In [23]:
pipeline.run()

X_train [[        nan -0.84055793  1.62953045 ... -0.30965005  1.07480248
   1.45379538]
 [ 0.12257962 -1.21890686  1.30988717 ...  0.37219356         nan
  -0.52682483]
 [-0.48137476  1.7220532  -0.97493091 ...  1.62583611 -0.88867034
   1.64376773]
 ...
 [-0.28096423  1.23419567 -0.55557567 ...  1.74939752  1.103659
  -1.00006082]
 [-0.1985477  -0.54601632 -1.05790815 ...  0.15352005 -0.06949833
  -0.06902909]
 [-1.46243996  0.54210308  1.16223638 ...  0.75850236 -1.14315444
   1.21787909]]
X_test [[-0.95474885 -0.70421016  0.47188818 ... -1.55665997         nan
   0.63773877]
 [ 1.75224249 -1.16749122 -0.53716073 ...  0.50682524  0.10058477
          nan]
 [ 1.21314573 -1.37832216 -0.47357335 ... -1.52052483  0.693906
  -0.54158247]
 ...
 [        nan  1.59651339 -0.33301916 ...         nan  0.4358387
   1.44637002]
 [ 1.165774    0.55912172 -0.96899383 ...  0.75284277 -0.72091983
   1.2775956 ]
 [-1.54236951 -0.99507004  1.5569424  ...  0.78099708 -0.50625657
          nan]]
delete